<a href="https://colab.research.google.com/github/ItalianPepper/VAE-GAN/blob/master/easy_vaegan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Some reference

- VAEGAN original : https://arxiv.org/pdf/1512.09300.pdf
- VAEGAN implementation https://github.com/crmaximo/VAEGAN
- Inspiring during implementation : https://towardsdatascience.com/10-lessons-i-learned-training-generative-adversarial-networks-gans-for-a-year-c9071159628
- Linneaus 5 (dataset): http://chaladze.com/l5/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzip a limited number of file
#!unzip -Z1 "*link" | head -10001 | sed 's| |\\ |g' | xargs unzip "*link" -d ./
# Linnaeus is .rar -> extract manually and transformed in .zip
!unzip "/content/drive/My Drive/Linnaeus 5 64X64.zip"

In [ ]:
!mkdir "/content/drive/My Drive/result"

In [ ]:
import time
import matplotlib.pyplot as plt
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import *


def get_dataset(path, img_shape):
    paths = []

    valid_format = [".jpg", ".png", ".tiff", ".bmp", ".gif"]

    for img in os.listdir(path):

        extension = os.path.splitext(img)[1]

        if extension.lower() in valid_format:
            paths.append(os.path.join(path, img))

    k_train = []

    for path_image in paths:
        img = load_img(path_image, target_size=img_shape[:2])

        el = img_to_array(img) / 255.0

        k_train.append(el)

    k_train = np.asarray(k_train)

    return k_train


def get_noise(n_sample=1, nlatent_dim=512):
    noise = np.random.normal(0, 1, (n_sample, nlatent_dim))

    return noise


def plot_generated_images(noise, nsample=1, path_save=None, epoch=0, name=None):
    imgs = decoder.predict(noise)
    img = imgs[0]

    # Solution: "Clipping input data to the valid range for imshow
    # with RGB data ([0..1] for floats or [0..255] for integers)."

    img = (img * 255).astype(np.uint8)

    fig = plt.figure(figsize=(40, 10))
    fig.patch.set_visible(False)
    epoch_str = str(epoch)
    extension = ".png"
    if name != None:
        path_name = path_save + "/" + name + "_" + epoch_str + extension
    else:
        path_name = path_save + "/" + epoch_str + extension

    ax = fig.add_subplot(1, nsample, 1)
    ax.imshow(img)
    ax.axis("off")

    # fig.suptitle("Epoch:" + epoch_str, fontsize=30)

    plt.savefig(path_name,
                bbox_inches='tight',
                pad_inches=0)

    plt.close()


def build_encoder(img_shape):
    input_img = Input(shape=img_shape, name="input_encoder")
    n_filter = 32

    # Encoding
    x = Conv2D(n_filter, kernel_size=(3, 3), padding="same")(input_img)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPooling2D(padding='same')(x)

    x = Conv2D(n_filter * 4, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPooling2D(padding='same')(x)

    x = Conv2D(n_filter * 8, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPooling2D(padding='same')(x)

    x = Flatten()(x)
    logsigma = Dense(512, activation="tanh")(x)

    model = Model(inputs=input_img, outputs=logsigma)
    # model.summary()
    return (model)


def build_decoder_gen(en_shape=(512,)):
    # impostazione del generatore
    en_img = Input(shape=en_shape, name="input_decoder")
    img_dim = 32 * 16
    n_filter = 32

    x = Dense(img_dim * 8 * 8)(en_img)
    x = Reshape((8, 8, img_dim))(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)

    # Decoding
    x = Conv2D(n_filter * 4, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D()(x)

    x = Conv2D(n_filter * 2, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D()(x)

    x = Conv2D(n_filter, kernel_size=(3, 3), padding="same")(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = UpSampling2D()(x)

    x = Conv2D(3, kernel_size=(3, 3), padding="same")(x)
    x = Activation('tanh')(x)

    model = Model(inputs=en_img, outputs=x)
    return (model)


def build_discriminator(img_shape):
    input_img = Input(shape=img_shape, name="input_discriminator")
    n_filter = 32

    x = Conv2D(n_filter, kernel_size=(3, 3), padding='same', strides=2)(input_img)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(n_filter * 2, kernel_size=(3, 3), padding='same', strides=2)(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(n_filter * 4, kernel_size=(3, 3), padding='same', strides=2)(x)
    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(128, kernel_size=(3, 3), padding='same', strides=2)(x)

    x = BatchNormalization(epsilon=1e-5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)

    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_img, outputs=out)

    return model


def train(encoder, decoder, discriminator, gan, x_train,
          dir_result="./", epochs=10000, batch_size=512, checkpoint=10):

    history = []

    for epoch in range(epochs):

        start_epoch = time.time()

        np.random.shuffle(x_train)

        batches_index = [x for x in range(0, len(x_train), batch_size)]

        for i in range(0, len(batches_index)):

            start = batches_index[i]

            if i == len(batches_index) - 1:
                end = len(x_train)
            else:
                end = batches_index[i + 1]

            x_batch = x_train[start:end]
            x_batch_size = len(x_batch)

            matrix_true = np.ones((x_batch_size, 1))
            matrix_false = np.zeros((x_batch_size, 1))

            # Prediction
            gen_img_enc = encoder.predict(x_batch)
            gen_img_dec = decoder.predict(gen_img_enc)

            noise = get_noise(n_sample=x_batch_size)
            fake_img = decoder.predict(noise)

            # Training

            dsc_real = discriminator.train_on_batch(x_batch, matrix_true)
            dsc_true_gen = discriminator.train_on_batch(gen_img_dec, matrix_true)
            dsc_fake = discriminator.train_on_batch(fake_img, matrix_false)

            dsc_true = 0.5*(dsc_real[0] + dsc_true_gen[0])

            decoder_loss = decoder.train_on_batch(noise, x_batch)

            gan_loss = gan.train_on_batch(x_batch, matrix_true)

        if (epoch + 1) % checkpoint == 0:
            history.append({"Epoch:": epoch + 1,
                            "Discriminator_loss_true:": dsc_true,
                            "Discriminator_loss_fake:": dsc_fake[0],
                            "Discriminator_acc_fake:": dsc_fake[1],
                            "Decoder_loss:": decoder_loss,
                            "GAN_loss:": gan_loss
                            })

            noise = get_noise(n_sample=1)
            plot_generated_images(noise, 1, dir_result, epoch + 1)

        end_epoch = time.time()
        end_epoch = end_epoch - start_epoch
        print("Ended Epoch {:0.0f}/{:1.0f} in: {:2.4f} s".format(epoch + 1, epochs, end_epoch))

    return history


start = time.time()

img_shape = (64, 64, 3)
noise_shape = (512,)
optimizer_std = SGD(0.0003)
optimizer_disc = SGD(0.0004)

pre_trained = False
path_load_models = "result/"

if pre_trained is False:
    encoder = build_encoder(img_shape)
    decoder = build_decoder_gen(noise_shape)
    discriminator = build_discriminator(img_shape)

    encoder.compile(loss="mse", optimizer=optimizer_std)
    decoder.compile(loss="mse", optimizer=optimizer_std)

    discriminator.compile(loss="mse", optimizer=optimizer_disc,
                          metrics=["accuracy"])

    img = Input(shape=img_shape, dtype="float32")
    encoded_img = encoder(img)
    decoded_img = decoder(encoded_img)
    val = discriminator(decoded_img)

    gan = Model(inputs=[img], outputs=[val])
    gan.compile(loss="mse", optimizer=optimizer_std)

else:
    encoder = load_model(path_load_models + "encoder.h5")
    decoder = load_model(path_load_models + "decoder.h5")
    discriminator = load_model(path_load_models + "discriminator.h5")
    gan = load_model(path_load_models + "gan.h5")

x_train = get_dataset("./Linnaeus 5 64X64/train/flower", img_shape)

path = "result/"
n_epoch = 1000
n_batch = 64
with tf.device("/gpu:0"):
    history = train(encoder, decoder, discriminator, gan, x_train,
                    dir_result=path, epochs=n_epoch, batch_size=n_batch,
                    checkpoint=40)

with open(path + "log_gan.txt", "w") as f:
    f.write("Epoch:" + str(n_epoch) + ", Batch Size:" + str(n_batch) + "\n")
    for item in history:
        item_str = str(item) + "\n"
        f.write(item_str)

# test post training
for i in range(0, 5):
    noise = get_noise(n_sample=1)

    plot_generated_images(noise, 1, path, i, name="post_training")

encoder.save(path+"encoder.h5", save_format="tf")
decoder.save(path + "decoder.h5", save_format="tf")
discriminator.save(path + "discriminator.h5", save_format="tf")
gan.save(path + "gan.h5", save_format="tf")

end = time.time()
end = end - start
print("Finished in: " + str(end) + " s")

In [ ]:
!rm -r ./result

In [ ]:
!rm -r ./Linnaeus\ 5\ 64X64